In [16]:
import requests
import numpy as np
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import time

In [17]:
def get_conn_postgres(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [18]:
def get_conn_readreplica(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [19]:
def get_df_from_sql_postgres(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [20]:
def get_df_from_sql_readreplica(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [21]:
def make_request(address, city):
    url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address'
    headers = {'Content-Type': 'application/json'}
    data = {
        "address": address,
        "city": city
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        
        response_json = response.json()
        
        # Extract lat_lng from the response JSON
        lat_lng = response_json.get('lat_lng', None)
        return lat_lng if lat_lng else None
        
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        print(f"Response content: {response.content}")
        return None

In [21]:
data = {
    'address': ['HYDERABAD Flat 106 Ashoka Spring Fields Kailash Nagar Ameenpur road Chandanagar Nursery before apartments on main road'],
    'city': ['Hyderabad']
}
df = pd.DataFrame(data)

# Get latitude and longitude for each address
df['lat_lng'] = df.apply(lambda row: make_request(row['address'], row['city']), axis=1)

print(df)

                                             address       city  \
0  HYDERABAD Flat 106 Ashoka Spring Fields Kailas...  Hyderabad   

                    lat_lng  
0  [17.5023033, 78.3174086]  


In [22]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = "select * from public.warehouses_warehouse where warehouse_city = 'Bangalore'"

# Establish a connection
conn = get_conn_readreplica(SSH_required, key_path)

# Retrieve data into a DataFrame
df_warehouse = get_df_from_sql_readreplica(SSH_required, query, key_path)
# df_warehouse = df_warehouse.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_warehouse)

Tunnel Started
Connection Made
     warehouse_int_id                                       warehouse_id  \
0                1467                                             Source   
1                1486                                                RK6   
2                  14                                       WE_Bangalore   
3                1507                                                 QA   
4                1543                                           Damensch   
..                ...                                                ...   
401              2039                                             Subrat   
402              2057  15 Ground Floor, SY No  131 3, Hoskote, Anjane...   
403              2058  Survey No  1 1B 1 2P  Gangadharanaplya Village...   
404              2088  Survey No  15 Ground Floor, SY No  131 3, Hosk...   
405              2090  Survey No  15 Ground Floor, SY No  131 3, Hosk...   

                                        warehouse_name  

In [25]:
query = "select * from application_db.node where city_name = 'Bangalore' and node_name <> 'BLR_OFSC'"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_hubs = get_df_from_sql_postgres(SSH_required, query, key_path)
# df_hubs = df_hubs.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_hubs)

df_active_hubs = pd.read_excel('active_hubs.xlsx')
print(df_active_hubs)

# Merge the dataframes based on the 'Node Name' column
active_hubs = pd.merge(df_hubs, df_active_hubs, left_on='node_name', right_on='Node Name', how='inner')

# Drop the 'Node Name' column as it's duplicated
active_hubs.drop(columns=['Node Name', 'Count'], inplace=True)

# Print the result
print(active_hubs)

Tunnel Started
Connection Made
    node_id  warehouse_id       node_name  location_id      team_names  \
0        18           182  UUTR-Franchise            1    BLR_FRH_UUTR   
1        31           182  STNG-Franchise            1    BLR_FRH_STNG   
2        12           182  NGWR-Franchise            1    BLR_FRH_NGWR   
3         6           146      GSCAN TEST            1    test_lm_hub1   
4         2           182     DS BLR BOMM            2          blr-lm   
5         1           182      DS BLR MTH            1          blr-lm   
6        10           182    BLR FRH ECTY            1    BLR_FRH_ECTY   
7         7           182     DS BLR HBBL            1  GS_BLR_LM_HBBL   
8        33           182  BLDR-Franchise            1    BLR_FRH_BLDR   
9        32           182  CMRJ-Franchise            1    BLR_FRH_CMRJ   
10       35           182  YLHK-Franchise            1    BLR_FRH_YLHK   

    fuel_rate  distance_limit sort_codes  \
0           0               0   BLR/

In [34]:
warehouse_lat_long_list = []

for index, row in df_warehouse.iterrows():
    lat_long = make_request(row['warehouse_address'], row['warehouse_city'])
    warehouse_lat_long_list.append(lat_long)
    # time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
df_warehouse['lat_long'] = warehouse_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(df_warehouse)


DataFrame with lat_long:
     warehouse_int_id                                       warehouse_id  \
0                1467                                             Source   
1                1486                                                RK6   
2                  14                                       WE_Bangalore   
3                1507                                                 QA   
4                1543                                           Damensch   
..                ...                                                ...   
401              2039                                             Subrat   
402              2057  15 Ground Floor, SY No  131 3, Hoskote, Anjane...   
403              2058  Survey No  1 1B 1 2P  Gangadharanaplya Village...   
404              2088  Survey No  15 Ground Floor, SY No  131 3, Hosk...   
405              2090  Survey No  15 Ground Floor, SY No  131 3, Hosk...   

                                        warehouse_name       

In [26]:
hubs_lat_long_list = []

for index, row in active_hubs.iterrows():
    lat_long = make_request(row['address'], row['city_name'])
    hubs_lat_long_list.append(lat_long)
    # time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
active_hubs['lat_long'] = hubs_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(active_hubs)


DataFrame with lat_long:
   node_id  warehouse_id       node_name  location_id      team_names  \
0       18           182  UUTR-Franchise            1    BLR_FRH_UUTR   
1       31           182  STNG-Franchise            1    BLR_FRH_STNG   
2        2           182     DS BLR BOMM            2          blr-lm   
3        1           182      DS BLR MTH            1          blr-lm   
4       10           182    BLR FRH ECTY            1    BLR_FRH_ECTY   
5        7           182     DS BLR HBBL            1  GS_BLR_LM_HBBL   
6       33           182  BLDR-Franchise            1    BLR_FRH_BLDR   
7       32           182  CMRJ-Franchise            1    BLR_FRH_CMRJ   
8       35           182  YLHK-Franchise            1    BLR_FRH_YLHK   

   fuel_rate  distance_limit sort_codes  \
0          0               0   BLR/UTTR   
1          0               0   BLR/STNG   
2          3             100   BLR/JPNR   
3          3             100   BLR/MRTH   
4          0             100

In [37]:
# Save DataFrame to a CSV file
csv_file_path = 'warehouses_w_lat_lng_blr.csv'
df_warehouse.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Warehouse file saved as {csv_file_path}.")

Warehouse file saved as warehouses_w_lat_lng_blr.csv.


In [27]:
# Save DataFrame to a CSV file
csv_file_path = 'hubs_w_lat_lng_blr.csv'
active_hubs.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Hubs file saved as {csv_file_path}.")

Hubs file saved as hubs_w_lat_lng_blr.csv.
